## Detección de Objetos

Esta semana exploraremos algunas técnicas utilizadas para la detección de objetos en fotografías o videos.

La detección de objetos es una tecnología informática relacionada con la visión por computadora y el procesamiento de imágenes que se ocupa de detectar instancias de objetos semánticos de una determinada clase (como humanos, edificios o automóviles) en imágenes y videos digitales.

A diferencia de la clasificación de fotografías como lo hemos hecho anteriormente en este curso, ahora daremos un paso adicional al tratar de encontrar el objeto de interés en una imagen. Pueda que exista uno o varios objetos, por tanto, se deben realizar algunas modificaciones a las arquitecturas previamente estudiadas para soportar dicho requerimiento. 

<img src="img/od1.png" />

### Tecnicas Comunes 

Hoy en día es común asociar el reconocimiento de objetos con Deep Learning por la gran capacidad para clasificar imágenes y elementos de alta complejidad. Lo cierto es que esta subdisciplina a sufrido una evolución donde han surgido gran cantidad de métodos, no todos rápidos, ni eficientes.  
Algunos de los métodos más populares son:

-	Detección con Cascadas Haar (AdaBoost) – Viola & Jones (2001)
-	Histograma de Gradientes (HOG) – (1986-2005) – McConnell (1986), Dalal (2005)
-	Propuesta de Regiones
    -	R-CNN  - Girshick (2014)
    -	Fast R-CNN - - Girshick (2015)
    -	Faster R-CNN - Shaoqing (2014)
    -	Mask R-CNN – Girshick et al (2017)
    -	Cascade R-CNN – Jiangmiao et al, (2019)
-	Single-Shot Multibox Detector (SSD) – Liu (2016)
-	YOLO – Redmon (2016, 2017, 2018) v1,v2,v3 – Bochkovskiy (2020) v4

### Clasificacion, Deteccion y Segmentacion

La segmentación es un tipo de etiquetado en el que cada píxel de una imagen se etiqueta con conceptos determinados. Aquí, las imágenes completas se dividen en grupos de píxeles que luego se pueden etiquetar y clasificar, con el objetivo de simplificar una imagen o cambiar la forma en que se presenta una imagen al modelo, para que sea más fácil de analizar.

Los modelos de segmentación proporcionan el contorno exacto del objeto dentro de una imagen. Es decir, se proporcionan detalles píxel a píxel para un objeto dado, a diferencia de los modelos de clasificación, donde el modelo identifica lo que hay en una imagen, y los modelos de detección, que coloca un cuadro delimitador alrededor de objetos específicos.

<img src="img/od2.jpeg" />

### Region-based R-CNN

Premisas:

- un solo objeto de interes puede estar en una determinada region
- Las regiones propuestas son generadas por un algoritmo llamado **Selective Search**
- internamente tiene 2 redes neuronales convolucionales.Uno para los bounding boxes y otro para las clases.

Selective Search:

- 1) se generan varias regiones alrededor de la imagen.
- 2) se combinan pares de regiones basadas en similitudes.
- 3) esto se repite hasta que quede solo una region... o muy pocas

<img src="img/selective-search.png" />

- 4) de las regiones generadas se generan una lista de posibles ventanas de clasificacion. 

#### Propuestas de Regiones

- RCNN genera cerca de 2000 regiones por foto.
- Se mueve una ventana de a traves de las regiones para identificar el objeto. Se extraen features de la imagen con SVM
- Se utiliza IoU para evaluar las ventanas e identificar un objeto. Recordemos que un objeto solo puede estar una vez en un bounding box, entonces se selecciona el mejor IoU.

<img src="img/iou.png" />


### Fast R-CNN


- Fast R-CNN usa solo una CNN para todos los features.
- Se sigue utilizando selective search para las regiones de interes (RoI)
- El RoI es una capa de pooling espacial piramidal.
- al final, la Fast  R-CNN se bifurca en 2 salidas: una con un softmax para las clases y otra con un regresor para los bounding boxes.
- ya no se usa SVM para la clasificacion de objetos, sino que se usa softmax.
- Estos cambios permitieron a Fast R-CNN ser mucho mas rapida para clasificar que R-CNN.

<img src="img/fast.png" />


### Faster R-CNN

- Se elimina Selective Search porque es costoso. 
- Se usa Region Proposal Network (RPN); otro algoritmo para generar regiones de forma mas eficiente.
- El resto de la arquitectura es igual a Fast R-CNN. 
- Basicamente la ganancia fue en el cambio por RPN.

<img src="img/faster.png" />

#### RPN:

Region Proposal Network toma una imagen de cualquier tamaño como entrada y genera un conjunto de propuestas de objetos rectangulares, cada una con una puntuación. Lo hace deslizando una pequeña red sobre el mapa de características generado por la capa convolucional.

Le features a partir de RPN se alimenta a dos capas hermanas completamente conectadas: una capa de regresión de cuadro para el cuadro delimitador y una capa de clasificación de cuadro para la clasificación de objetos.

Los **Arnchors** son los tamanos de los square features que se usan para evaluar el contenido de la ventana que se mueve (sliding window).

<img src="img/anchors.png" />



### Yolo v3:

<img src="img/yolov3.png" />

Yolo aplica una única red neuronal a la imagen completa. Esta red divide la imagen en regiones y predice cuadros delimitadores y probabilidades para cada región. Estos cuadros delimitadores están ponderados por las probabilidades predichas.

El modelo tiene varias ventajas sobre los sistemas basados en clasificadores. Mira la imagen completa en el momento de la prueba, por lo que sus predicciones se basan en el contexto global de la imagen. También hace predicciones con una única evaluación de red, a diferencia de sistemas como R-CNN, que requieren miles para una sola imagen. Esto lo hace extremadamente rápido, más de 1000 veces más rápido que R-CNN y 100 veces más rápido que Fast R-CNN. 

Caracteristicas

- genera un grid por toda la imagen. Este grid es el equivalente a las regiones. por tanto usa menos regiones que la mayoria de los algoritmos de deteccion.
- cada uno de las regiones solamente puede ser asignado a una clase.
- Yolo v3 esta compuesto por 106 capas.
- Al igual que RetinaNet, Yolo es un one-shot detector.

<img src="img/yolomap.jpg" />

### Metricas

#### IoU: Intersection over union (Ap-IoU)

IoU mide la superposición entre 2 límites. Usamos eso para medir cuánto se superpone nuestro límite predicho con la verdad fundamental (el límite del objeto real). En algunos conjuntos de datos, predefinimos un umbral de IoU (digamos 0,5) para clasificar si la predicción es un verdadero positivo o un falso positivo.


#### mAP: mean Average Precision 

AP (precisión promedio) es una métrica popular para medir la precisión de detectores de objetos como Faster R-CNN, SSD, etc. La precisión promedio calcula el valor de precisión promedio para el valor de Recall de 0 a 1.

Recall: TP / (TP+FN)


### Velocidad de los detectores de objetos

<img src="img/yolo.png" />

<img src="img/yolot.png" />

Yolo v4: https://arxiv.org/pdf/2004.10934.pdf


### Formatos de Etiquetado & Herramientas

#### PASCAL VOC XML

http://host.robots.ox.ac.uk/pascal/VOC/#:~:text=The%20PASCAL%20VOC%20project%3A,2005%2D2012%2C%20now%20finished)

<img src="img/voc.png" />

herramientas:

- https://github.com/manhcuogntin4/Label-Annotation-VOC-Pascal
- https://github.com/wkentaro/labelme
- https://rectlabel.com/

### COCO Labels

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch
https://cocodataset.org/#format-data

<img src="img/coco.png" />

herramientas:

- https://github.com/wkentaro/labelme
- https://github.com/jsbroks/coco-annotator
- https://github.com/visipedia/annotation_tools

### YOLO Labels

<img src="img/yoloa.png" />

herramientas:

- https://github.com/tzutalin/labelImg
